In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql import SQLContext

import wget
import os
from datetime import datetime

In [2]:
!mkdir /home/ebbcf/datasets/

mkdir: cannot create directory ‘/home/ebbcf/datasets/’: File exists


In [ ]:
!mkdir /home/ebbcf/logs/

In [4]:
filepath = '/home/ebbcf/datasets/' 

In [5]:
logpath = '/home/ebbcf/logs/' 

In [ ]:
json_part_files = [
'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00000.json.gz',
'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00001.json.gz',
'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00002.json.gz',
'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00003.json.gz',
'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00004.json.gz',
'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00005.json.gz',
'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00006.json.gz',
'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00007.json.gz',
'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00008.json.gz',
'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00009.json.gz']

In [ ]:
etapa1 = {}

In [ ]:
for json in json_part_files:  
    wget.download(json, out=filepath + json.split("/")[4])

In [ ]:
sql = """
 SELECT COUNT(*)
 FROM df_temp ses
 WHERE NOT EXISTS (SELECT 1
                      FROM df_temp other
                      WHERE other.anonymous_id = ses.anonymous_id
                        AND  other.device_sent_timestamp >= ses.device_sent_timestamp - 1800000
                        AND other.device_sent_timestamp < ses.device_sent_timestamp 
                   )
"""


In [ ]:
df = spark.read.json('file:///home/ebbcf/datasets/part-00000.json.gz')

In [ ]:
df.createOrReplaceTempView('df_temp')

In [ ]:
df.show()

In [ ]:
import datetime
timestamp_with_ms = 1592616733778 -1800000

dt = datetime.datetime.fromtimestamp(timestamp_with_ms / 1000)
formatted_time = dt.isoformat(sep=' ', timespec='milliseconds')

print(formatted_time)

In [ ]:
timestamp_with_ms = 1592616733778 

dt = datetime.datetime.fromtimestamp(timestamp_with_ms / 1000)
formatted_time = dt.isoformat(sep=' ', timespec='milliseconds')
print(formatted_time)

In [ ]:
def unique_sessions(file, sql):
    df = spark.read.json('file:///home/ebbcf/datasets/' + file)
    df.createOrReplaceTempView('df_temp')
    spark.catalog.cacheTable('df_temp')
    unique_sessions = spark.sql(sql).collect()
    spark.catalog.uncacheTable('df_temp')
    return unique_sessions[0][0]

In [ ]:
from datetime import datetime

In [ ]:
for file in os.listdir(filepath):
    if file[0:4] == 'part':
        etapa1[file] = unique_sessions(file, sql)  
        f = open(logpath + 'log' + datetime.now().strftime("%d_%m_%Y") + '.txt', "a")
        f.write("Finalmente tem log: " + str(etapa1) + '\n')
        f.close()

In [ ]:
etapa1

In [9]:
sessionby = ['browser_family', 'os_family', 'device_family']

In [6]:
df = spark.read.json('file://' + filepath)

In [ ]:
df.createOrReplaceTempView('df_temp')

In [ ]:
 spark.catalog.cacheTable('df_temp')

In [13]:
etapa2 = {}

In [11]:
df.write.parquet('hdfs:///user/bartolomeu/sessions2.parquet')

In [14]:
for type_session in sessionby:
    df = spark.sql('SELECT ' + type_session + ', COUNT(*) qtd ' +
                    ' FROM parquet.`hdfs:///user/bartolomeu/sessions2.parquet` ses ' +
                    ' WHERE NOT EXISTS (SELECT 1 ' +
                    ' FROM parquet.`hdfs:///user/bartolomeu/sessions2.parquet` other ' +
                    ' WHERE other.anonymous_id = ses.anonymous_id ' + 
                    '   AND  other.device_sent_timestamp >= ses.device_sent_timestamp - 1800000 ' +
                     '   AND other.device_sent_timestamp < ses.device_sent_timestamp) GROUP BY ' + type_session)
    etapa2[type_session] = {r[type_session]: r['qtd'] for r in df.collect()}

Py4JJavaError: An error occurred while calling o201.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 27.0 failed 1 times, most recent failure: Lost task 4.0 in stage 27.0 (TID 789, localhost, executor driver): org.apache.spark.memory.SparkOutOfMemoryError: error while calling spill() on org.apache.spark.shuffle.sort.ShuffleExternalSorter@2d29ca20 : No space left on device
	at org.apache.spark.memory.TaskMemoryManager.acquireExecutionMemory(TaskMemoryManager.java:219)
	at org.apache.spark.memory.TaskMemoryManager.allocatePage(TaskMemoryManager.java:285)
	at org.apache.spark.memory.MemoryConsumer.allocatePage(MemoryConsumer.java:117)
	at org.apache.spark.shuffle.sort.ShuffleExternalSorter.acquireNewPageIfNecessary(ShuffleExternalSorter.java:377)
	at org.apache.spark.shuffle.sort.ShuffleExternalSorter.insertRecord(ShuffleExternalSorter.java:401)
	at org.apache.spark.shuffle.sort.UnsafeShuffleWriter.insertRecordIntoSorter(UnsafeShuffleWr

In [ ]:
etapa2

Etapa 2 Completa!

In [ ]:
etapa3 = {}

In [ ]:
df = spark.read.json('file://' + filepath)

In [ ]:
df.write.parquet(filepath + 'sessions3.parquet')

In [ ]:
for type_session in sessionby:
    df_session = spark.sql('SELECT '+ type_session + ', anonymous_id, (max(device_sent_timestamp) - min(device_sent_timestamp)) as duration from parquet.`'+ filepath + 'sessions3.parquet` group by '+ type_session + ', anonymous_id')
    df_session.createOrReplaceTempView('df_session_temp')
    df = spark.sql('select '+ type_session + ', format_number(percentile(duration, 0.5), 2) mediana from df_session_temp group by '+ type_session)                   
    etapa3[type_session] = {r[type_session]: r['mediana'] for r in df.collect()}

In [ ]:
etapa3

Etapa 3 Finalizada!